In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

In [ ]:
pip install -qU "langchain[google-genai]"

In [ ]:
pip install -qU langchain-core

In [ ]:
pip install pypdf

In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
from google.colab import files
uploaded=files.upload()

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Your custom fallback message
custom_prompt = ChatPromptTemplate.from_messages([
    ("system", "Use the following context to answer the question. "
               "If you don't know the answer, say: 'I don’t know. I have no available information.'"
               "If someone ask you about banana you can use all information on internet to help about bananas"
               "When you feel like it you can ask is there anythink i can help you with and if he say no you can response only with 'bye' which i can use in while loop to break it"),
    ("human", "Question: {question}\n\nContext:\n{context}")
])

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load the PDF
loader = PyPDFLoader("primjer.pdf")
docs = loader.load()

# Split the text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

prompt = custom_prompt


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [ ]:
while True:
    question = input("Enter your question (or type 'exit' to quit): ")
    if question.lower() == "exit":
        break
    response = graph.invoke({"question": question})
    print(response["answer"])
    if response["answer"] =="bye":
        break